In [1]:
import thirdai
from thirdai import bolt_v2 as bolt, dataset
import numpy as np

In [25]:
input_dim = 200
num_examples = 5000

In [26]:
def chunk(array, chunksize):
    chunks = []
    for i in range(0, len(array), chunksize):
        chunks.append(np.array(array[i : i + chunksize]))
    return chunks

In [27]:
def generate_data_labels(input_dim, num_examples):
    data = np.random.randint(0, input_dim, size=num_examples, dtype="uint32")
    labels_np = data

    labels = bolt.train.convert_dataset(
        dataset.from_numpy(
            labels_np, batch_size = 100
        ),
        dim = input_dim
    )

    data = bolt.train.convert_dataset(
        dataset.from_numpy(
            data, batch_size = 100
        ),
        dim = input_dim
    )
    return data, labels, chunk(labels_np, 100)

In [28]:
def generate_embedding_model(use_robez, input_dim, output_dim):
    token_input = bolt.nn.Input(dim=input_dim)

    if use_robez:
        hidden = bolt.nn.RobeZ(
            num_embedding_lookups=8,
            lookup_size=64,
            log_embedding_block_size=29,
            update_chunk_size=8,
            reduction="sum",
        )(token_input)
    else:
        hidden = bolt.nn.PosEmbedding(
            num_embedding_lookups=8,
            lookup_size=64,
            log_embedding_block_size=29,
            update_chunk_size=8,
            reduction="sum",
        )(token_input)
        
    output = bolt.nn.FullyConnected(
        dim=output_dim, input_dim=hidden.dim(), activation="softmax"
    )(hidden)

    labels = bolt.nn.Input(dim=output_dim)
    loss = bolt.nn.losses.CategoricalCrossEntropy(activations=output, labels=labels)

    model = bolt.nn.Model(
        inputs=[token_input],
        outputs=[output],
        losses=[loss],
    )
    return model

In [29]:
def train(model, train_data, train_labels, learning_rate, epochs):
    for _ in range(epochs):
        for x,y in zip(train_data, train_labels):
            model.train_on_batch(x,y)
            model.update_parameters(learning_rate)

In [30]:
def evaluate(model, test_data, test_labels_np):
    correct = 0
    total = 0
    for x, y_np in zip(test_data, test_labels_np):
        output = model.forward(x, use_sparsity=False)

        correct += np.sum(np.argmax(output[0].activations, axis=1) == y_np)
        total += len(y_np)
    print(f"Accuracy: {correct/total}")

In [31]:
robezmodel = generate_embedding_model(True, input_dim = input_dim, output_dim = input_dim)
train_data, train_labels, label_chunks = generate_data_labels(input_dim, num_examples)
test_data, _, test_labels_np = generate_data_labels(input_dim, num_examples)

In [32]:
train(robezmodel, train_data, train_labels, 0.0001, 3)
evaluate(robezmodel, test_data, test_labels_np)

Accuracy: 0.9962


In [33]:
posmodel = generate_embedding_model(False, input_dim = input_dim, output_dim = input_dim)
train_data, train_labels, label_chunks = generate_data_labels(input_dim, num_examples)
test_data, _, test_labels_np = generate_data_labels(input_dim, num_examples)

In [35]:
train(posmodel, train_data, train_labels, 0.0001, 3)
evaluate(posmodel, test_data, test_labels_np)

Accuracy: 0.9954
